In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import shapely
from scipy.spatial import distance

In [3]:
# Chemins des fichiers
#lake_shapefile = "E:/Mémoire master fidel/Thème_2/WQI/SIG/Shp/Lake_Aheme.shp"
shp_path = "E:/Mémoire master fidel/Thème_2/WQI/SIG/Shp/Lake_Aheme.shp"
lake_shapefile = gpd.read_file(shp_path).to_crs(epsg=32631)
data_Aheme = pd.read_excel("Data_Aheme.xlsx", sheet_name="Data_Aheme")

C:\Users\INGENIEUR\anaconda3\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


In [4]:
data_Aheme

,Date,Mois_jr,PP,Tmoy,Season,Saison,Heure,Code,Stations,X,...,Turbidity,Chl_a,N_NO2,N_NH4,N_NO3,P_PO4,PT,NT,Long,Lat
0,2023-10-05,10-05,161.28,28.0,PSP,PSP,09:05:00,1,A1,1.93618,...,67.86,NaN,8.23,150.40,0.17,167.98,250.98,22.36,382769.630037,706304.353377
1,2023-10-05,10-05,161.28,28.0,PSP,PSP,09:31:00,1,A2,1.95130,...,49.66,NaN,12.47,131.76,0.16,169.26,233.10,24.59,384024.157078,708590.427515
2,2023-10-05,10-05,161.28,28.0,PSP,PSP,10:00:00,1,A3,1.96690,...,33.14,NaN,10.21,104.36,0.20,190.97,230.55,24.96,385759.620745,713573.256780
3,2023-10-05,10-05,161.28,28.0,PSP,PSP,10:31:00,1,A4,1.99330,...,32.76,NaN,7.38,86.83,0.14,166.71,222.89,26.40,388689.340098,718664.272053
4,2023-10-05,10-05,161.28,28.0,PSP,PSP,10:58:00,1,A5,1.98400,...,31.56,NaN,10.49,71.48,0.20,153.94,192.24,20.31,387667.342624,721850.495347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2024-10-10,10-10,159.78,27.4,PSP,PSP,11:03:00,13,A6,1.98220,...,15.88,9.61,30.65,35.95,0.14,199.77,410.58,6.58,387475.180794,725245.137356
126,2024-10-10,10-10,159.78,27.4,PSP,PSP,11:16:00,13,A7,1.98090,...,40.14,3.20,63.85,39.90,0.13,158.07,409.30,6.24,387338.017558,728473.832198
127,2024-10-10,10-10,159.78,27.4,PSP,PSP,12:18:00,13,A8,1.96550,...,15.02,5.34,32.96,37.36,0.11,198.23,392.70,10.19,385612.262295,717244.223525
128,2024-10-10,10-10,159.78,27.4,PSP,PSP,12:39:00,13,A9,1.93730,...,18.96,7.48,28.33,39.34,0.21,181.24,395.25,11.10,382486.591631,713823.225992


# Analyse spatiale

In [11]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import shapely
from scipy.spatial import distance
from shapely.vectorized import contains
import os

def create_multiparameter_map(data_Aheme, lake_shapefile, parameters, output_dir='./results', dpi=300):
    """
    Crée et sauvegarde des cartes multi-paramètres avec un fichier par paramètre
    Chaque paramètre contient 13 mois sur une image de 7 colonnes × 2 lignes
    
    Parameters:
    ----------
    data_Aheme : DataFrame
        Données contenant les colonnes X, Y, Date et les colonnes des paramètres à visualiser
    lake_shapefile : str
        Chemin vers le shapefile du lac
    parameters : dict
        Dictionnaire des paramètres à visualiser avec leurs unités {nom_paramètre: unité}
    output_dir : str
        Répertoire de sauvegarde
    dpi : int
        Résolution pour les images
    """
    try:
        # Créer le répertoire de sortie s'il n'existe pas
        os.makedirs(output_dir, exist_ok=True)
        print(f"Sauvegarde dans le répertoire: {os.path.abspath(output_dir)}")
        
        # 1. Chargement du shapefile du lac et reprojection
        lake = gpd.read_file(lake_shapefile).to_crs(epsg=32631)
        
        # Augmenter de 5% la délimitation du lac
        xmin, ymin, xmax, ymax = lake.total_bounds
        buffer = (xmax - xmin) * 0.05
        xmin -= buffer
        xmax += buffer
        ymin -= buffer
        ymax += buffer

        # 2. Conversion CRS des données
        gdf = gpd.GeoDataFrame(
            data_Aheme,
            geometry=gpd.points_from_xy(data_Aheme.X, data_Aheme.Y),
            crs="EPSG:4326"
        ).to_crs(lake.crs)

        data_Aheme = data_Aheme.copy()
        data_Aheme["X"] = gdf.geometry.x
        data_Aheme["Y"] = gdf.geometry.y

        # 3. Prétraitement temporel
        data_Aheme["Date"] = pd.to_datetime(data_Aheme["Date"], format="%d/%m/%Y")
        data_Aheme['Month'] = data_Aheme['Date'].dt.strftime('%b %y').str.upper()
        
        # Obtenir tous les mois uniques et les trier chronologiquement
        all_months = sorted(data_Aheme['Month'].unique())
        print(f"Mois disponibles: {all_months}")
        
        # Vérifier qu'on a bien 13 mois
        if len(all_months) != 13:
            print(f"⚠ Attention: {len(all_months)} mois trouvés au lieu de 13")
        
        # 4. Grille d'interpolation
        grid_resolution = 300
        grid_x, grid_y = np.mgrid[xmin:xmax:grid_resolution*1j, ymin:ymax:grid_resolution*1j]
        
        # Masque du lac optimisé
        lake_poly = lake.geometry.union_all()
        points_x = grid_x.flatten()
        points_y = grid_y.flatten()
        mask = contains(lake_poly, points_x, points_y).reshape(grid_x.shape)
        
        # 5. Traiter chaque paramètre séparément
        for param_name, param_unit in parameters.items():
            print(f"\nTraitement du paramètre: {param_name}")
            
            # Filtrer les données pour ce paramètre
            data_filtered = data_Aheme.dropna(subset=[param_name])
            
            if len(data_filtered) == 0:
                print(f"  Aucune donnée pour {param_name}")
                continue
                
            param_min = data_filtered[param_name].min()
            param_max = data_filtered[param_name].max()
            print(f"  Plage de valeurs: {param_min:.2f} - {param_max:.2f} {param_unit}")
            
            # Créer une seule figure avec 7 colonnes et 2 lignes pour les 13 mois
            n_cols = 7
            n_rows = 2
            
            # Taille de la figure adaptée à 7×2
            fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))
            
            # Aplatir les axes pour faciliter l'itération
            axes_flat = axes.flatten()
            
            # Boucle sur les 13 mois
            for month_idx, month in enumerate(all_months):
                if month_idx >= len(axes_flat):  # Sécurité si plus de 14 mois
                    break
                    
                ax = axes_flat[month_idx]
                month_data = data_filtered[data_filtered['Month'] == month]
                
                if len(month_data) < 3:
                    ax.text(0.5, 0.5, 'Données\ninsuffisantes', 
                           ha='center', va='center', transform=ax.transAxes, fontsize=10)
                    ax.set_title(f"{month}", fontsize=12, pad=10, weight='bold')
                    ax.axis('off')
                    continue
                
                # Interpolation IDW
                coords = month_data[['X', 'Y']].values
                values = month_data[param_name].values
                points = np.vstack((grid_x.flatten(), grid_y.flatten())).T
                
                # Calcul des distances et poids
                dist_matrix = distance.cdist(points, coords)
                dist_matrix[dist_matrix == 0] = 1e-10
                weights = 1 / dist_matrix**2
                grid_z_idw_flat = np.sum(weights * values, axis=1) / np.sum(weights, axis=1)
                grid_z_idw = grid_z_idw_flat.reshape(grid_x.shape)
                
                # Application du masque
                masked_data = np.where(mask, grid_z_idw, np.nan)
                
                # Tracé avec normalisation pour ce paramètre
                norm = Normalize(vmin=param_min, vmax=param_max)
                im = ax.pcolormesh(grid_x, grid_y, masked_data, cmap='Spectral_r', norm=norm, shading='auto')
                lake.boundary.plot(ax=ax, color='black', linewidth=0.8)
                
                # Titre du mois
                ax.set_title(f"{month}", fontsize=12, pad=10, weight='bold')
                ax.set_aspect('equal')
                ax.axis('off')
            
            # Masquer les axes inutilisés s'il y a moins de 14 mois
            for i in range(len(all_months), len(axes_flat)):
                axes_flat[i].axis('off')
            
            # Ajouter la barre de couleur pour ce paramètre
            cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
            cbar = fig.colorbar(im, cax=cbar_ax, orientation='vertical')
            cbar.set_label(f'{param_name} ({param_unit})', fontsize=14, rotation=270, labelpad=20)
            
            # Titre principal pour cette figure
            fig.suptitle(f"Distribution spatiale de {param_name}\nOctobre 2023 à Octobre 2024", 
                        fontsize=16, y=0.98, weight='bold')
            
            # Ajuster la mise en page
            plt.tight_layout(rect=[0, 0, 0.9, 0.96])
            
            # Sauvegarde pour ce paramètre
            filename = f"{param_name}_13_mois.png"
            filepath = os.path.join(output_dir, filename)
            plt.savefig(filepath, dpi=dpi, bbox_inches='tight', facecolor='white')
            print(f"  Sauvegardé: {filename}")
            
            plt.close(fig)  # Fermer la figure pour libérer la mémoire
        
        print(f"\n✓ Tous les paramètres ont été sauvegardés dans: {os.path.abspath(output_dir)}")
        
    except Exception as e:
        print(f"Erreur lors de la création des cartes: {e}")
        import traceback
        traceback.print_exc()

# Exemple d'utilisation:
if __name__ == "__main__":
    # Chargement des données
    lake_shapefile = r"E:\Mémoire master fidel\Thème_2\WQI\SIG\Shp\Lake_Aheme.shp"
    data_Aheme = pd.read_excel("Data_Aheme.xlsx", sheet_name="Data_Aheme")

    # Définition des paramètres à visualiser avec leurs unités
    parameters = {
        'Temperature': '°C',
        'Conductivity': 'ms/cm',
        'Salinity': 'psu',
        'O2': 'mg/L',
        'Saturation': '%',
        'pH': '',
        'Redox': 'mV',
        'Transparence': 'cm',
        'Turbidity': 'NTU',
        'Chl_a': 'µg/L',
        'N_NO2': 'µg/L',
        'N_NH4': 'µg/L',
        'N_NO3': 'mg/L',
        'P_PO4': 'µg/L',
        'PT': 'µg/L',
        'NT': 'mg/L'
    }
    
    # Appel de la fonction avec sauvegarde par paramètre
    create_multiparameter_map(
        data_Aheme=data_Aheme,
        lake_shapefile=lake_shapefile,
        parameters=parameters,
        output_dir='./cartes_parametres',  # Répertoire de sauvegarde
        dpi=600  # Haute résolution
    )

Sauvegarde dans le répertoire: C:\Users\INGENIEUR\Documents\Memoire\cartes_parametres
Mois disponibles: ['APR 24', 'AUG 24', 'DEC 23', 'FEB 24', 'JAN 24', 'JUL 24', 'JUN 24', 'MAR 24', 'MAY 24', 'NOV 23', 'OCT 23', 'OCT 24', 'SEP 24']

Traitement du paramètre: Temperature
  Plage de valeurs: 25.00 - 33.00 °C


C:\Users\INGENIEUR\AppData\Local\Temp\ipykernel_3564\833742968.py:157: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.96])


  Sauvegardé: Temperature_13_mois.png

Traitement du paramètre: Conductivity
  Plage de valeurs: 0.11 - 48.10 ms/cm


C:\Users\INGENIEUR\AppData\Local\Temp\ipykernel_3564\833742968.py:157: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.96])


  Sauvegardé: Conductivity_13_mois.png

Traitement du paramètre: Salinity
  Plage de valeurs: 0.00 - 31.70 psu


...

  Sauvegardé: P_PO4_13_mois.png

Traitement du paramètre: PT
  Plage de valeurs: 63.85 - 704.24 µg/L


C:\Users\INGENIEUR\AppData\Local\Temp\ipykernel_3564\833742968.py:157: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.96])


  Sauvegardé: PT_13_mois.png

Traitement du paramètre: NT
  Plage de valeurs: 0.18 - 37.91 mg/L


C:\Users\INGENIEUR\AppData\Local\Temp\ipykernel_3564\833742968.py:157: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.96])


  Sauvegardé: NT_13_mois.png

✓ Tous les paramètres ont été sauvegardés dans: C:\Users\INGENIEUR\Documents\Memoire\cartes_parametres
